## Labels

The values of a categorical variable are selected from a group of categories, also called labels. Thus, in the variable _gender_ the categories or labels are male and female, whereas in the variable _city_ the labels can be London, Manchester, Brighton and so on.

We can see from the above examples, that different categorical variables can contain different amount of labels (or categories). The variable gender contains only 2 labels, but a variable like city or postcode, can contain a huge number of different labels.

The number of labels within a categorical variable is known as cardinality. A high number of labels within a variable is known as __high cardinality__.

### Are multiple labels in a categorical variable a problem?

If highly cardinal categorical variables are used to train machine learning models, the following problems may appear: 

- Variables with too many labels tend to dominate over those with only a few labels, particularly in **Tree based** algorithms.

- A big number of labels within a variable may introduce noise with little if any information, therefore making the machine learning models prone to over-fit.

- Some of the labels may only be present in the training data set, but not in the test set, therefore causing the machine learning algorithms to over-fit the training set.

- Contrarily, new labels may appear in the test set that were not present in the training set, therefore leaving the machine learning algorithm unable to perform a calculation over the new observation.


In particular, **tree methods are biased towards variables with lots of labels**, so their performance may be affected by these type of variables.

Below, I will show the effect of high cardinality of variables on the performance of different machine learning algorithms, and how a quick fix to reduce the number of labels, without any sort of data insight, already helps to boost performance.



## Real Life example: 

### Predicting Survival on the Titanic: understanding society behaviour and beliefs

Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

====================================================================================================

To download the Titanic data, go ahead to this [website](https://www.kaggle.com/c/titanic/data)

Click on the link 'train.csv', and then click the 'download' blue button towards the right of the screen, to download the dataset. Save it in a folder of your choice.

**Note that you need to be logged in to Kaggle in order to download the datasets**.

If you save it in the same directory from which you are running this notebook, and you rename the file to 'titanic.csv' then you can load it the same way I will load it below.

====================================================================================================

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [7]:
# let's load the titanic dataset

data = pd.read_csv('titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The categorical variables in this dataset are Name, Sex, Ticket, Cabin and Embarked.

In [11]:
# let's inspect at the number of labels for the different categorical variables

print('Number of categories in the variable Name: {}'.format(len(data.Name.unique())))

print('Number of categories in the variable Gender: {}'.format(
    len(data.Sex.unique())))

print('Number of categories in the variable Ticket: {}'.format(
    len(data.Ticket.unique())))

print('Number of categories in the variable Cabin: {}'.format(
    len(data.Cabin.unique())))

print('Number of categories in the variable Embarked: {}'.format(
    len(data.Embarked.unique())))

print('Total number of passengers in the Titanic: {}'.format(len(data)))

Number of categories in the variable Name: 891
Number of categories in the variable Gender: 2
Number of categories in the variable Ticket: 681
Number of categories in the variable Cabin: 148
Number of categories in the variable Embarked: 4
Total number of passengers in the Titanic: 891


While the variable Sex contains only 2 categories and Embarked 4, the variables Ticket and Cabin, as expected, contain a huge number of different labels. They show high cardinality. And so does the variable Name (different for each passenger).

To demonstrate the effect of high cardinality in machine learning performance, I will work with the variable Cabin.

In [12]:
# let's explore the values / categories of Cabin

# we know from the previous cell that there are 148 different cabins
# therefore the variable is highly cardinal

data.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

Let's now reduce the cardinality of the variable. How? instead of using the entire Cabin value, I will capture only the 
first letter.

***Rationale***: the first letter indicates the deck on which the cabin was located, and is therefore an indication of both social class status and proximity to the surface of the Titanic. Both are known to help or improve the probability of survival.

In [16]:
data['Cabin_reduced'] = data['Cabin'].astype(str).str[0]
data[['Cabin_reduced','Cabin']].head()

,Cabin_reduced,Cabin
0,n,NaN
1,C,C85
2,n,NaN
3,C,C123
4,n,NaN


In [6]:
print('Number of categories in the variable Cabin: {}'.format(
    len(data.Cabin.unique())))

print('Number of categories in the variable Cabin reduced: {}'.format(
    len(data.Cabin_reduced.unique())))

Number of categories in the variable Cabin: 148
Number of categories in the variable Cabin reduced: 9


We reduced the number of different labels from 148 to 9.

In [17]:
# let's separate into training and testing set
# in order to build machine learning models

use_cols = ['Cabin', 'Cabin_reduced', 'Sex']

X_train, X_test, y_train, y_test = train_test_split(
    data[use_cols], #only columns will be used in the model
    data.Survived,  #this is the column of test data, 
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((623, 3), (268, 3))

### High cardinality leads to uneven distribution of categories in train and test sets

When a variable is highly cardinal, often some categories land only on the training set, or only on the testing set. If present only in the training set, they may lead to overfitting. If present only on the testing set, the machine learning algorithm will not know how to handle them, as it has not seen it during training.

In [18]:
# Let's find out labels present only in the training set

unique_to_train_set = [
    x for x in X_train.Cabin.unique() if x not in X_test.Cabin.unique()
]

len(unique_to_train_set)

100

There are 100 Cabins only present in the training set, and not in the testing set.

In [19]:
# Let's find out labels present only in the test set

unique_to_test_set = [
    x for x in X_test.Cabin.unique() if x not in X_train.Cabin.unique()
]

len(unique_to_test_set)

28

Variables with high cardinality tend to have values (i.e., categories) present in the training set, that are not present in the test set, and vice versa. This will bring problems at the time of training (due to over-fitting) and scoring of new data (how should the model deal with unseen categories?).

In order to evaluate the effect of categorical variables in machine learning models, I will quickly replace the categories by numbers. See below.

In [20]:
# Let's re-map Cabin into numbers so we can use it to train ML models

# I will replace each cabin by a number
# This will allow me to quickly demonstrate
# the effect of labels on machine learning algorithms

cabin_dict = {k:i for i, k in enumerate(data.Cabin.unique(), 0)}
cabin_dict

{nan: 0,
 'C85': 1,
 'C123': 2,
 'E46': 3,
 'G6': 4,
 'C103': 5,
 'D56': 6,
 'A6': 7,
 'C23 C25 C27': 8,
 'B78': 9,
 'D33': 10,
 'B30': 11,
 'C52': 12,
 'B28': 13,
 'C83': 14,
 'F33': 15,
 'F G73': 16,
 'E31': 17,
 'A5': 18,
 'D10 D12': 19,
 'D26': 20,
 'C110': 21,
 'B58 B60': 22,
 'E101': 23,
 'F E69': 24,
 'D47': 25,
 'B86': 26,
 'F2': 27,
 'C2': 28,
 'E33': 29,
 'B19': 30,
 'A7': 31,
 'C49': 32,
 'F4': 33,
 'A32': 34,
 'B4': 35,
 'B80': 36,
 'A31': 37,
 'D36': 38,
 'D15': 39,
 'C93': 40,
 'C78': 41,
 'D35': 42,
 'C87': 43,
 'B77': 44,
 'E67': 45,
 'B94': 46,
 'C125': 47,
 'C99': 48,
 'C118': 49,
 'D7': 50,
 'A19': 51,
 'B49': 52,
 'D': 53,
 'C22 C26': 54,
 'C106': 55,
 'C65': 56,
 'E36': 57,
 'C54': 58,
 'B57 B59 B63 B66': 59,
 'C7': 60,
 'E34': 61,
 'C32': 62,
 'B18': 63,
 'C124': 64,
 'C91': 65,
 'E40': 66,
 'T': 67,
 'C128': 68,
 'D37': 69,
 'B35': 70,
 'E50': 71,
 'C82': 72,
 'B96 B98': 73,
 'E10': 74,
 'E44': 75,
 'A34': 76,
 'C104': 77,
 'C111': 78,
 'C92': 79,
 'E38': 80,
 'D

In [21]:
# replace the labels in Cabin, using the dic created above
X_train.loc[:, 'Cabin_mapped'] = X_train.loc[:, 'Cabin'].map(cabin_dict)
X_test.loc[:, 'Cabin_mapped'] = X_test.loc[:, 'Cabin'].map(cabin_dict)

X_train[['Cabin_mapped', 'Cabin']].head(10)

,Cabin_mapped,Cabin
857,143,E17
52,10,D33
386,0,NaN
124,20,D26
578,0,NaN
549,0,NaN
118,22,B58 B60
12,0,NaN
157,0,NaN
127,0,NaN


We see how NaN takes the value 2 in the new variable, E17 takes the value 0, D33 takes the value 1, and so on.

In [22]:
# Now I will replace the letters in the reduced cabin variable

# create replace dictionary
cabin_dict = {k: i for i, k in enumerate(X_train['Cabin_reduced'].unique(), 0)}

# replace labels by numbers with dictionary
X_train.loc[:, 'Cabin_reduced'] = X_train.loc[:, 'Cabin_reduced'].map(cabin_dict)
X_test.loc[:, 'Cabin_reduced'] = X_test.loc[:, 'Cabin_reduced'].map(cabin_dict)

X_train[['Cabin_reduced', 'Cabin']].head(10)

,Cabin_reduced,Cabin
857,0,E17
52,1,D33
386,2,NaN
124,1,D26
578,2,NaN
549,2,NaN
118,3,B58 B60
12,2,NaN
157,2,NaN
127,2,NaN


We see now that D33 and D26 take the same number, 1, because we are capturing only the letter. They both start with D.

In [23]:
# re-map the categorical variable Sex into numbers

X_train.loc[:, 'Sex'] = X_train.loc[:, 'Sex'].map({'male': 0, 'female': 1})
X_test.loc[:, 'Sex'] = X_test.loc[:, 'Sex'].map({'male': 0, 'female': 1})

X_train.Sex.head()

857    0
52     1
386    0
124    0
578    1
Name: Sex, dtype: int64

In [24]:
# check if there are missing values in these variables

X_train[['Cabin_mapped','Cabin_reduced', 'Sex']].isnull().sum()

Cabin_mapped     0
Cabin_reduced    0
Sex              0
dtype: int64

In [25]:
X_test[['Cabin_mapped','Cabin_reduced', 'Sex']].isnull().sum()

Cabin_mapped     0
Cabin_reduced    0
Sex              0
dtype: int64

In the test set, there are now 30 missing values for the highly cardinal variable. These were introduced when encoding the categories into numbers. How? Many categories exist only in the test set. Thus, when we creating our encoding dictionary using only the train set, we did not generate a number to replace those labels present only in the test set. As a consequence, they were encoded as NaN. We will see in future lectures how to tackle this problem. For now, I will fill those missing values with 0.

In [26]:
# let's check the number of categories in the encoded variables
len(X_train.Cabin_mapped.unique()), len(X_train.Cabin_reduced.unique())

(121, 9)

We can see how we reduced the number of different categories from 148 to just 9 in our previous step. Let's go ahead and evaluate the effect of labels in machine learning algorithms.

### Random Forests

In [27]:
# model built on data with high cardinality for cabin

# call the model
rf = RandomForestClassifier(n_estimators=200, random_state=39)

# train the model
rf.fit(X_train[['Cabin_mapped', 'Sex']], y_train)

# make predictions on train and test set
pred_train = rf.predict_proba(X_train[['Cabin_mapped', 'Sex']])
pred_test = rf.predict_proba(X_test[['Cabin_mapped', 'Sex']].fillna(0))

print('Train set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Random Forests roc-auc: 0.8617329342096701
Test set
Random Forests roc-auc: 0.8093154761904762


We observe that the performance of the Random Forests on the training set is quite superior to its performance in the test set. This indicates that the model is over-fitting, which means that it does a great job at predicting the outcome on the dataset it was trained on, but it lacks the power to generalise the prediction to unseen data.

In [28]:
# model built on data with low cardinality for cabin

# call the model
rf = RandomForestClassifier(n_estimators=200, random_state=39)

# train the model
rf.fit(X_train[['Cabin_reduced', 'Sex']], y_train)

# make predictions on train and test set
pred_train = rf.predict_proba(X_train[['Cabin_reduced', 'Sex']])
pred_test = rf.predict_proba(X_test[['Cabin_reduced', 'Sex']])

print('Train set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Random Forests roc-auc: 0.8199550985878832
Test set
Random Forests roc-auc: 0.8332142857142857


We can see now that the Random Forests no longer over-fit to the training set. In addition, the model is much better at generalising the predictions (compare the roc-auc of this model on the test set vs the roc-auc of the model above also in the test set: 0.83 vs 0.80).

### AdaBoost

In [29]:
# model build on data with plenty of categories in Cabin variable

# call the model
ada = AdaBoostClassifier(n_estimators=200, random_state=44)

# train the model
ada.fit(X_train[['Cabin_mapped', 'Sex']], y_train)

# make predictions on train and test set
pred_train = ada.predict_proba(X_train[['Cabin_mapped', 'Sex']])
pred_test = ada.predict_proba(X_test[['Cabin_mapped', 'Sex']].fillna(0))

print('Train set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Adaboost roc-auc: 0.8388755124617687
Test set
Adaboost roc-auc: 0.8396726190476189


In [30]:
# model build on data with fewer categories in Cabin Variable

# call the model
ada = AdaBoostClassifier(n_estimators=200, random_state=44)

# train the model
ada.fit(X_train[['Cabin_reduced', 'Sex']], y_train)

# make predictions on train and test set
pred_train = ada.predict_proba(X_train[['Cabin_reduced', 'Sex']])
pred_test = ada.predict_proba(X_test[['Cabin_reduced', 'Sex']].fillna(0))

print('Train set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Adaboost roc-auc: 0.8195863430294354
Test set
Adaboost roc-auc: 0.8332142857142857


Similarly, the Adaboost model trained on the variable with high cardinality is slightly overfit to the train set. Whereas the Adaboost trained on the low cardinal variable is not overfitting and it does a better job in generalising the predictions.

In addition, building an AdaBoost on a model with less categories in Cabin, is a) simpler and b) should a different category in the test set appear, by taking just the front letter of cabin, the ML model will know how to handle it because it was seen during training.

### Logistic Regression

In [31]:
# model build on data with plenty of categories in Cabin variable

# call the model
logit = LogisticRegression(random_state=44)

# train the model
logit.fit(X_train[['Cabin_mapped', 'Sex']], y_train)

# make predictions on train and test set
pred_train = logit.predict_proba(X_train[['Cabin_mapped', 'Sex']])
pred_test = logit.predict_proba(X_test[['Cabin_mapped', 'Sex']].fillna(0))

print('Train set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Logistic regression roc-auc: 0.8173304266718727
Test set
Logistic regression roc-auc: 0.834345238095238


C:\Users\Bot1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [32]:
# model build on data with fewer categories in Cabin Variable

# call the model
logit = LogisticRegression(random_state=44)

# train the model
logit.fit(X_train[['Cabin_reduced', 'Sex']], y_train)

# make predictions on train and test set
pred_train = logit.predict_proba(X_train[['Cabin_reduced', 'Sex']])
pred_test = logit.predict_proba(X_test[['Cabin_reduced', 'Sex']].fillna(0))

print('Train set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Logistic regression roc-auc: 0.7672664367367301
Test set
Logistic regression roc-auc: 0.7957738095238095


C:\Users\Bot1\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


We can draw the same conclusion for Logistic Regression. Reducing the cardinality improves the performance and generalisation of the algorithm.

### Gradient Boosted Classifier

In [33]:
# model build on data with plenty of categories in Cabin variable

# call the model
gbc = GradientBoostingClassifier(n_estimators=300, random_state=44)

# train the model
gbc.fit(X_train[['Cabin_mapped', 'Sex']], y_train)

# make predictions on train and test set
pred_train = gbc.predict_proba(X_train[['Cabin_mapped', 'Sex']])
pred_test = gbc.predict_proba(X_test[['Cabin_mapped', 'Sex']].fillna(0))

print('Train set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Gradient Boosted Trees roc-auc: 0.8711036636949308
Test set
Gradient Boosted Trees roc-auc: 0.8156249999999999


In [24]:
# model build on data with plenty of categories in Cabin variable

# call the model
gbc = GradientBoostingClassifier(n_estimators=300, random_state=44)

# train the model
gbc.fit(X_train[['Cabin_reduced', 'Sex']], y_train)

# make predictions on train and test set
pred_train = gbc.predict_proba(X_train[['Cabin_reduced', 'Sex']])
pred_test = gbc.predict_proba(X_test[['Cabin_reduced', 'Sex']].fillna(0))

print('Train set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Gradient Boosted Trees roc-auc: 0.8204756946703976
Test set
Gradient Boosted Trees roc-auc: 0.8332142857142857


Gradient Boosted trees are indeed over-fitting to the training set in those cases where the variable Cabin has a lot of labels. This was expected as tree methods tend to be biased to variables with plenty of categories.

**That is all for this demonstration. I hope you enjoyed the notebook, and see you in the next one.**